<a href="https://colab.research.google.com/github/ellozam/ciencia-datos-notebooks/blob/main/proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Instala las librerías necesarias (solo una vez)
!pip install openml gradio pandas seaborn matplotlib

import openml
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gradio as gr

def filtrar_dataset(openml_id, min_gdp, max_gdp, country, year_start, year_end, education_level, employment_rate):
    try:
        # Cargar el dataset por ID
        dataset = openml.datasets.get_dataset(openml_id)
        X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute)

        # Mostrar las columnas del dataset para saber con qué trabajar
        print(f"Columnas en el dataset: {X.columns}")

        # Filtrar el dataset por condiciones múltiples
        filtered = X[(y > min_gdp) & (y < max_gdp)]  # Filtrar por GDP
        if country:
            filtered = filtered[filtered['country'].str.contains(country, case=False, na=False)]  # Filtrar por país (ignora mayúsculas/minúsculas)
        if year_start and year_end:
            filtered = filtered[(filtered['year'] >= year_start) & (filtered['year'] <= year_end)]  # Filtrar por año
        if education_level:
            filtered = filtered[filtered['education_level'].str.contains(education_level, case=False, na=False)]  # Filtrar por nivel de educación
        if employment_rate:
            filtered = filtered[filtered['employment_rate'] >= employment_rate]  # Filtrar por tasa de empleo

        # Generar gráficos
        fig, ax = plt.subplots(1, 2, figsize=(14, 6))

        # Gráfico de dispersión entre PIB per cápita y nivel de educación
        sns.scatterplot(data=filtered, x='gdp_per_capita', y='education_level', ax=ax[0])
        ax[0].set_title('PIB per cápita vs Nivel de Educación')

        # Gráfico de dispersión entre PIB per cápita y tasa de empleo
        sns.scatterplot(data=filtered, x='gdp_per_capita', y='employment_rate', ax=ax[1])
        ax[1].set_title('PIB per cápita vs Tasa de Empleo')

        # Mostrar gráficos
        plt.tight_layout()
        plt.show()

        # Mostrar las primeras filas del dataframe filtrado
        return filtered.head(20)

    except Exception as e:
        return f"Error: {e}"

# Interfaz de Gradio
gr.Interface(
    fn=filtrar_dataset,
    inputs=[
        gr.Number(label="ID del dataset en OpenML", value=45104),  # Cambié a un dataset económico
        gr.Number(label="PIB mínimo", value=1000),
        gr.Number(label="PIB máximo", value=50000),
        gr.Textbox(label="País (opcional)", value="USA"),  # Campo de texto para país
        gr.Number(label="Año de inicio", value=2000),  # Año de inicio
        gr.Number(label="Año de fin", value=2020),  # Año de fin
        gr.Textbox(label="Nivel de educación (opcional)", value="High School"),  # Filtro por educación
        gr.Number(label="Tasa de empleo mínima", value=50),  # Tasa de empleo mínima
    ],
    outputs=[gr.Dataframe(label="Resultados filtrados")],
    title="Filtrado de Datasets Económicos con Gráficos (OpenML + Gradio)",
    description="Filtra un dataset económico por PIB, país, nivel de educación, tasa de empleo y muestra gráficos interactivos."
).launch()




Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e9f0e0e2f5866762b5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
